In [ ]:
def lambda_for_normal_A(n,theta_0, sigma_0, theta_a, sigma_a):
    '''
    n = sample size,
    theta_0 = mean of the null hypthesis,
    sigma_0 = variance of the null hypothesis,
    theta_a = mean of the sample,
    sigma_a = varaince of the sample.
    This test is working under asumption that the null data is normally distributed around the null mean and the 
    alternative hypothesis is distributed as normal as well. 
    '''
    fraction = (np.sqrt(sigma_a/(sigma_a + n*sigma_0)))
    result = fraction*np.exp(((n**2)*sigma_0*((theta_a - theta_0)**2))/(2*sigma_a*(sigma_a + (n*sigma_0))))
    return result

def lambda_for_normal_AB(n,theta_0, sigma_0, theta_AB, sigma_AB):
    '''
    n = sample size,
    theta_0 = mean of the null hypthesis,
    sigma_0 = variance of the null hypothesis,
    theta_AB = mean of the difference between test A and test B.
    sigma_AB = varaince of the difference between test A and test B.
    This test is working under asumption that the null data is normally distributed around the null mean and the 
    alternative hypothesis is distributed as normal as well. 
    '''
    fraction = (np.sqrt((2*sigma_AB) / (2*sigma_AB + n*sigma_0)))
    result = fraction*np.exp(((n**2)*sigma_0*((theta_AB - theta_0)**2))/(4*sigma_AB*(2*sigma_AB + (n*sigma_0))))
    return result


def lambda_for_binary(n,theta_0, sigma_0, theta_A):
    '''
    n = sample size,
    theta_0 = mean of the null hypthesis,
    sigma_0 = variance of the null hypothesis,
    theta_A = mean of the test A sample,
    asymptotically distributed as N(theta, V/n) hence it is simple to evaluate the mSPRT.
    '''
    V = (theta_A*(1 - theta_A)) #The variance
    fraction = np.sqrt((V) / (V + (n*sigma_0)))
    result = fraction*np.exp(((n**2)*sigma_0*((theta_A - theta_0)**2))/(2*V*(V + (n*sigma_0))))
    return result

def valid_p_values(p,result):
    '''p is the list of previous p values and result is a float. New list of P values is returned'''
    if np.array(p).min() > (1/result):
        p.append(1/result)
    else:
        pass
    return p

# working example:

To do:
- improve the prior by either using a more appropriate distribution or using an infinitley weak prior
- More intelligent parameter approximation

In [ ]:
storage = dict()
regions_other = dict()
regions_base = dict()
for variant in experiment.variants:
    samples = np.array([int(len(dfA['s_converted'])),int(len(dfB['s_converted']))]).min()
    step = 1000
    size = int(np.ceil(samples / step))
    result = np.zeros(size)
    pval = [1]
    x = dataframe1['s_total_margin']
    y = dataframe2['s_total_margin']
    X = [x[0:1000].mean()]
    Y = [y[0:1000].mean()]
    i = 0
    n = 1000
    while i <= 99:
        n += 1000
        i += 1 
        X.append(x[(n):(n+1000)].mean())
        Y.append(y[(n):(n+1000)].mean())
        theta_0 = np.mean(X)
        sigma_0 = np.var(X)
        theta_a = np.mean(Y)
        sigma_a = np.var(Y)
        result[int(n/step)] = lambda_for_normal_A(int(n/step),theta_0, sigma_0, theta_a, sigma_a)
        pval = valid_p_values(pval,result[int(n/step)])
        storage[f'{experiment.id_}: A compared with {variant}'] = (n,i,np.array(pval).min())
        if np.array(pval).min() < 0.05:

To veiw the results in a more understanable way, from here you can plot what ever you wish.

In [ ]:
dff = pd.DataFrame();dff['res'] = result;dff['SS'] = 1 - pval;dff['maxSS'] = dff['SS'].max();dff['const'] = 0.95
dff.tail()